In [1]:
import tensorflow as tf
#import tensorflow.nn.rnn_cell as rnn
import numpy as np
#import tensorflow.keras as keras
#from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K


from tqdm import tqdm
import os

In [2]:
#load 100hz data
data_x=np.loadtxt('./PSDSData (35)_raw.csv',delimiter=",",dtype=np.float32)
#data_y=np.loadtxt('./Tap(24)_y_100Hz.csv',delimiter=",",dtype=np.float32)
#preY = np.loadtxt('./Tap(24)_y_100Hz.csv',delimiter=",",dtype=np.float32)
#data_x=data_x[:,3:9];
data_y=np.zeros(len(data_x))
data_x.shape,data_y.shape

((3582, 13), (3582,))

In [3]:
#reduce from 100hz to 25hz
# new_x=np.zeros_like(data_x[0:1])
# new_y=np.zeros_like(data_y[0:1])
# for i in range(int(len(data_x)/4)):
#     temp=data_x[ i*4 : (i+1)*4 ].mean(axis=0)
#     for j in range(len(temp)):
#         if temp[j] >= 0:
#             temp[j]=data_x[ i*4 : (i+1)*4 ].max(axis=0)[j]
#         else:
#             temp[j]=data_x[ i*4 : (i+1)*4 ].min(axis=0)[j]
#     new_x=np.append(new_x,[temp],axis=0)
#     new_y=np.append(new_y,data_y[i*4:(i+1)*4].max())
# new_x=np.delete(new_x,0,axis=0)
# new_y=np.delete(new_y,0,axis=0)


input_data=data_x[:,3:9]
input_data[:,3:6]=np.arccos(input_data[:,3:6]/10)
x_states=np.zeros([1,128],dtype=np.float32)
output_data=np.zeros(len(input_data))

input_data.shape,x_states.shape,output_data.shape

((3582, 6), (1, 128), (3582,))

In [4]:
interpreter=tf.lite.Interpreter(model_path="keras_acos_1024.tflite")
#interpreter=tf.lite.Interpreter(model_path="tap_9_24.tflite")
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()

In [5]:
def updateInputHistory(input):
    for i in range(InputHistorySize-1,0,-1):
        inputHistory[i]=inputHistory[i-1]
    inputHistory[0]=input
        
def updatePredictions(pred):
    for i in range(PredictionHistorySize-1,0,-1):
        predictionHistory[i]=predictionHistory[i-1]
    predictionHistory[0]=pred


In [6]:
def verify(taps,label_taps):
    False_positive= np.zeros_like(taps)
    False_negtive = np.zeros_like(taps)
    True_positive = np.zeros_like(taps)
    window=5
    #taps=np.append(np.zeros([1]),taps)
    #taps=np.append(taps,np.zeros([1]))
    #label_taps=np.append(np.zeros([1]),label_taps)
    #label_taps=np.append(label_taps,np.zeros([1]))
    for i in range(len(taps)):
        if label_taps[i] ==1:
            if taps[i-window:i+window].sum()==0:
                False_negtive[i]=1
            else:
                True_positive[i]=1

        if taps[i] ==1:
            if label_taps[i-window:i+window].sum()==0:
                False_positive[i]=1
                #raw_buff=raw[i-4:i+1,2]# only z acc
                #diff=abs(raw_buff.max()-raw_buff.min())
                #print(i,raw_buff,diff)
            
    return [False_positive,  False_negtive ,    True_positive ]

In [7]:
def didPush(i,lastPush,pushGap):
    if lastPush != -1:
        timeDiff=i-lastPush
        if timeDiff < TimeDiffThreshold:
            push=0
            return push,lastPush
    
    if (inputHistory[InputHistorySize-1] is None) or (predictionHistory[PredictionHistorySize-1] is None):
        push=0
        return push,lastPush

    if all(pred > predictionThreshold for pred in predictionHistory) & pushGap:
        push=1
        lastPush=i
    else:
        push=0
    return push,lastPush

In [11]:
TimeDiffThreshold=15
predictionThreshold=0.2

InputHistorySize=4
inputHistory=[None]*InputHistorySize
PredictionHistorySize=1
predictionHistory=[None]*PredictionHistorySize
output_push=[None]*len(input_data)

In [12]:
lastPush=-1
pushGap=True
for i in tqdm(range(len(input_data))):
    input_x=input_data[i,:].reshape([1,6])
    updateInputHistory(input_x)
    
    interpreter.set_tensor(input_details[1]['index'],x_states)
    interpreter.set_tensor(input_details[0]['index'],input_x)
    interpreter.invoke()
    
    output_data[i]=interpreter.get_tensor(output_details[0]['index'])
    x_states=interpreter.get_tensor(output_details[1]['index'])
    
    updatePredictions(output_data[i])
     
    output_push[i],lastPush=didPush(i,lastPush,pushGap)
    
    if (lastPush >-1 ) & (output_data[i]<predictionThreshold):
        pushGap=True
    else:
        if lastPush==i:
            pushGap=False

100%|██████████| 3582/3582 [00:00<00:00, 55814.15it/s]


In [20]:
output_data.sum()/50

5.497448392808438

In [18]:
len(output_push)

3582

In [19]:
sum(output_push)

50

In [10]:
False_positive,  False_negtive ,    True_positive=verify(np.array(output_push),data_y)

False_positive.sum(),  False_negtive.sum() ,    True_positive.sum()

(1139, 0, 0)

In [16]:
np.savetxt('output_push.csv',output_push)

np.savetxt('fp.csv',False_positive)

np.savetxt('fn.csv',False_negtive)

np.savetxt('tp.csv',True_positive)

np.savetxt('output_data.csv',output_data)

In [13]:
np.savetxt('push_35_keras.csv',output_push)
np.savetxt('predict_35_keras.csv',output_data)